In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

In [3]:
import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

ts_info_fullfn = debris_prms.ts_fp + debris_prms.roi + '_debris_tsinfo.nc'

if os.path.exists(debris_prms.ts_fp) == False:
    os.makedirs(debris_prms.ts_fp)
    
debug = False

print(ts_info_fullfn)

/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/16_ts_data/16_debris_tsinfo.nc


In [4]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(debris_prms.mb_binned_fp):
    if i.endswith('mb_bins.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            if region < 10:
                rgiid_list.append(i[0:7])
            else:
                rgiid_list.append(i[0:8])
            rgiid_fn_list.append(i)
        

# rgiid_list = ['18.00718']
# rgiid_fn_list = ['18.00718_mb_bins.csv']

rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)



main_glac_rgi = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
main_glac_rgi['bin_fn'] = rgiid_fn_list

20 glaciers in region 16 are included in this model run: ['01339', '01346', '01350', '01909', '01963', '02137', '02172', '02322', '02348', '02394', '02410', '02412', '02420', '02433', '02457', '02482', '02483', '02497', '02518', '02566'] and more
This study is focusing on 20 glaciers in region [16]


In [5]:
# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_%'].values))
main_glac_rgi['DC_Area_%'] = main_glac_rgi.RGIId.map(dc_areaperc_dict).fillna(0)
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))
main_glac_rgi['DC_Area_v2'] = main_glac_rgi.RGIId.map(dc_area_dict).fillna(0)
main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,1333,RGI60-16.01339,-78.1446,-0.489511,16,1,9.487,4484,5692,5042,...,0,9,20019999,1339,16.01339,16.01339,281.8554,16.01339_mb_bins.csv,19.543,1768975
1,1340,RGI60-16.01346,-77.9845,0.033719,16,1,2.617,4380,5773,5181,...,0,9,20019999,1346,16.01346,16.01346,282.0155,16.01346_mb_bins.csv,21.632,520990
2,1344,RGI60-16.01350,-76.0329,2.913660,16,1,2.260,4493,5370,4856,...,0,9,20019999,1350,16.01350,16.01350,283.9671,16.01350_mb_bins.csv,7.447,137765
3,1903,RGI60-16.01909,-76.9059,-10.291500,16,1,6.223,4485,6317,5221,...,0,9,20080812,1909,16.01909,16.01909,283.0941,16.01909_mb_bins.csv,13.479,825962
4,1957,RGI60-16.01963,-77.2433,-9.960370,16,1,2.705,4713,5578,5073,...,0,9,20080812,1963,16.01963,16.01963,282.7567,16.01963_mb_bins.csv,27.549,744679
5,2130,RGI60-16.02137,-77.4339,-9.416480,16,1,5.090,4464,6090,5258,...,0,9,20080812,2137,16.02137,16.02137,282.5661,16.02137_mb_bins.csv,9.283,465745
6,2165,RGI60-16.02172,-77.3129,-9.551200,16,1,5.165,4520,5652,5107,...,0,9,20080812,2172,16.02172,16.02172,282.6871,16.02172_mb_bins.csv,6.151,266186
7,2315,RGI60-16.02322,-77.5191,-9.192290,16,1,4.050,4341,6153,5140,...,0,9,20031018,2322,16.02322,16.02322,282.4809,16.02322_mb_bins.csv,16.178,647294
8,2341,RGI60-16.02348,-77.4705,-9.275600,16,1,3.038,4349,6081,5104,...,0,9,20031018,2348,16.02348,16.02348,282.5295,16.02348_mb_bins.csv,19.760,594293


In [6]:
# Latitude and longitude index to run the model
#  Longitude must be 0 - 360 degrees
latlon_all = []
for i in os.listdir(debris_prms.ostrem_fp):
    if i.endswith(debris_prms.ostrem_fn_sample.split('XXXX')[1]):
        latlon_fn = i.split(debris_prms.ostrem_fn_sample.split('XXXX')[1])[0]
        # Extract latitude
        lat_str = latlon_fn.split('-')[0]
        if 'N' in lat_str:
            i_lat = int(lat_str.split('N')[0]) / 100
        elif 'S' in lat_str:
            i_lat = -1 * int(lat_str.split('S')[0]) / 100
        # Extract longitude
        lon_str = latlon_fn.split('-')[1]
        i_lon = int(lon_str.split('E')[0]) / 100
        latlon_all.append([i_lat, i_lon, i])
latlon_all = sorted(latlon_all)

lat_all = np.array([x[0] for x in latlon_all])
lon_all = np.array([x[1] for x in latlon_all])
ostrem_fn_all_raw = [x[2] for x in latlon_all]

main_glac_rgi['lat_nearest'] = np.nan
main_glac_rgi['lon_nearest'] = np.nan
main_glac_rgi['ostrem_fn'] = np.nan
for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
    latlon_dist = (((main_glac_rgi.loc[glac_idx,'CenLat'] - lat_all)**2 + 
                    (main_glac_rgi.loc[glac_idx,'CenLon_360'] - lon_all)**2)**0.5)
    latlon_nearidx = np.where(latlon_dist == latlon_dist.min())[0][0]
    
    main_glac_rgi.loc[glac_idx,'lat_nearest'] = lat_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'lon_nearest'] = lon_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'ostrem_fn'] = ostrem_fn_all_raw[latlon_nearidx]
    
ostrem_fn_all = sorted(list(np.unique(main_glac_rgi['ostrem_fn'].values)))
lat_values = np.arange(main_glac_rgi.lat_nearest.min(), main_glac_rgi.lat_nearest.max() + 0.1, 0.25)
lon_values = np.arange(main_glac_rgi.lon_nearest.min(), main_glac_rgi.lon_nearest.max() + 0.1, 0.25)

main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2,lat_nearest,lon_nearest,ostrem_fn
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,1333,RGI60-16.01339,-78.1446,-0.489511,16,1,9.487,4484,5692,5042,...,1339,16.01339,16.01339,281.8554,16.01339_mb_bins.csv,19.543,1768975,43.25,250.5,4325N-25050E-debris_melt_curve.nc
1,1340,RGI60-16.01346,-77.9845,0.033719,16,1,2.617,4380,5773,5181,...,1346,16.01346,16.01346,282.0155,16.01346_mb_bins.csv,21.632,520990,43.25,250.5,4325N-25050E-debris_melt_curve.nc
2,1344,RGI60-16.01350,-76.0329,2.913660,16,1,2.260,4493,5370,4856,...,1350,16.01350,16.01350,283.9671,16.01350_mb_bins.csv,7.447,137765,43.25,250.5,4325N-25050E-debris_melt_curve.nc
3,1903,RGI60-16.01909,-76.9059,-10.291500,16,1,6.223,4485,6317,5221,...,1909,16.01909,16.01909,283.0941,16.01909_mb_bins.csv,13.479,825962,43.25,250.5,4325N-25050E-debris_melt_curve.nc
4,1957,RGI60-16.01963,-77.2433,-9.960370,16,1,2.705,4713,5578,5073,...,1963,16.01963,16.01963,282.7567,16.01963_mb_bins.csv,27.549,744679,43.25,250.5,4325N-25050E-debris_melt_curve.nc
5,2130,RGI60-16.02137,-77.4339,-9.416480,16,1,5.090,4464,6090,5258,...,2137,16.02137,16.02137,282.5661,16.02137_mb_bins.csv,9.283,465745,43.25,250.5,4325N-25050E-debris_melt_curve.nc
6,2165,RGI60-16.02172,-77.3129,-9.551200,16,1,5.165,4520,5652,5107,...,2172,16.02172,16.02172,282.6871,16.02172_mb_bins.csv,6.151,266186,43.25,250.5,4325N-25050E-debris_melt_curve.nc
7,2315,RGI60-16.02322,-77.5191,-9.192290,16,1,4.050,4341,6153,5140,...,2322,16.02322,16.02322,282.4809,16.02322_mb_bins.csv,16.178,647294,43.25,250.5,4325N-25050E-debris_melt_curve.nc
8,2341,RGI60-16.02348,-77.4705,-9.275600,16,1,3.038,4349,6081,5104,...,2348,16.02348,16.02348,282.5295,16.02348_mb_bins.csv,19.760,594293,43.25,250.5,4325N-25050E-debris_melt_curve.nc


In [7]:
# Process each group and derive elevation statistics for the debris cover
year_mean = np.zeros((len(lat_values), len(lon_values)))
year_std = np.zeros((len(lat_values), len(lon_values)))
year_med = np.zeros((len(lat_values), len(lon_values)))
year_mad = np.zeros((len(lat_values), len(lon_values)))
doy_mean = np.zeros((len(lat_values), len(lon_values)))
doy_std = np.zeros((len(lat_values), len(lon_values)))
doy_med = np.zeros((len(lat_values), len(lon_values)))
doy_mad = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mean = np.zeros((len(lat_values), len(lon_values)))
dayfrac_std = np.zeros((len(lat_values), len(lon_values)))
dayfrac_med = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mad = np.zeros((len(lat_values), len(lon_values)))


ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)

for nlatlon, ostrem_fn in enumerate(ostrem_fn_all):
# for nlatlon, ostrem_fn in enumerate([ostrem_fn_all[96]]):
    
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['ostrem_fn'] == ostrem_fn]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    lat_idx = np.where(main_glac_rgi_subset.loc[0,'lat_nearest'] == lat_values)[0][0]
    lon_idx = np.where(main_glac_rgi_subset.loc[0,'lon_nearest'] == lon_values)[0][0]
    
    lat_deg = lat_values[lat_idx]
    lon_deg = lon_values[lon_idx]
    
    print(nlatlon, lat_deg, lon_deg, ostrem_fn)

    doy_list = []
    year_list = []
    dayfrac_list = []
#     print('\n\nDELETE ME: SWITCH BACK!\n\n')
    for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
        glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

#         print(nglac, glac_idx, rgiid)

        # Ts filenames
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        ts_year_fn = ts_fns_df.loc[ts_fn_idx,'ts_year_fullfn']
        ts_doy_fn = ts_fns_df.loc[ts_fn_idx,'ts_doy_fullfn']
        ts_dayfrac_fn = ts_fns_df.loc[ts_fn_idx,'ts_dayfrac_fullfn']

        
        # Process debris-covered glaciers
        if (((main_glac_rgi_subset.loc[glac_idx, 'DC_Area_%'] > debris_prms.dc_percarea_threshold) | 
             (main_glac_rgi_subset.loc[glac_idx, 'DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
            and '0.0' not in ts_fns_df.loc[ts_fn_idx].values):

            # Create glacier feature from ice thickness raster
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()
            
            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_dhdt=False, gf_add_vel=False, 
                          gf_add_ts=True, ts_fn=ts_fn,
                          gf_add_ts_info=True, ts_year_fn=ts_year_fn, ts_doy_fn=ts_doy_fn, ts_dayfrac_fn=ts_dayfrac_fn)

            # Isolate the debris-covered areas
            gf.ts = np.ma.array(gf.ts.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts.data)))
            gf.ts_dayfrac = np.ma.array(gf.ts_dayfrac.data, 
                                        mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_dayfrac.data)))
            gf.ts_year = np.ma.array(gf.ts_year.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_year.data)))
            gf.ts_doy = np.ma.array(gf.ts_doy.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_doy.data)))

#             # ===== PLOTS =====
#             # DEM
#             var_full2plot = gf.z1.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#             # Surface temperature
#             var_full2plot = gf.ts.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#             # Ts Day fraction
#             var_full2plot = gf.ts_dayfrac.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-dayfrac'], 'inferno', 'ts dayfrac', close_fig=False)
#             # Ts Year
#             var_full2plot = gf.ts_year.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-year'], 'inferno', 'ts year', close_fig=False)
#             # Ts Day of year
#             var_full2plot = gf.ts_doy.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-doy'], 'inferno', 'doy', close_fig=False)
            
#         # ========================================================================================================
                    
#             print(gf.ts_year.compressed())
#             print(gf.ts_doy.compressed())
    
            doy_list.extend(list(gf.ts_doy.compressed()))
            year_list.extend(list(gf.ts_year.compressed()))
            dayfrac_list.extend(list(gf.ts_dayfrac.compressed()))

    
    if len(doy_list) > 0 and len(year_list) > 0 and len(dayfrac_list) > 0:
        # Year
        year_mean_latlon = np.mean(year_list)
        year_std_latlon = np.std(year_list)
        year_med_latlon = malib.fast_median(year_list)
        year_mad_latlon = malib.mad(year_list)
        # Day of year
        #  adjust day of year to avoid issues with comparing JD 1 (Jan 1) and JD 365 (Dec 31),
        #  since the mean would be JD 182 (June), but should be JD 0.5 (Jan 1); also would affect std, etc.
        doy_array = np.array(doy_list)
        doy_array[doy_array > 365] = 365
        doy_array[doy_array > 182] = doy_array[doy_array > 182] - 365
        doy_mean_latlon = np.mean(doy_array)
        if doy_mean_latlon < 0:
            doy_mean_latlon += 365
        doy_std_latlon = np.std(doy_array)
        doy_med_latlon = malib.fast_median(doy_array)
        if doy_med_latlon < 0:
            doy_med_latlon += 365
        doy_mad_latlon = malib.mad(doy_array)
        dayfrac_mean_latlon = np.mean(dayfrac_list)
        dayfrac_std_latlon = np.std(dayfrac_list)
        dayfrac_med_latlon = malib.fast_median(dayfrac_list)
        dayfrac_mad_latlon = malib.mad(dayfrac_list)

        # Update array
        year_mean[lat_idx,lon_idx] = year_mean_latlon
        year_std[lat_idx,lon_idx] = year_std_latlon
        year_med[lat_idx,lon_idx] = year_med_latlon
        year_mad[lat_idx,lon_idx] = year_mad_latlon
        doy_mean[lat_idx,lon_idx] = doy_mean_latlon
        doy_std[lat_idx,lon_idx] = doy_std_latlon
        doy_med[lat_idx,lon_idx] = doy_med_latlon
        doy_mad[lat_idx,lon_idx] = doy_mad_latlon
        dayfrac_mean[lat_idx,lon_idx] = dayfrac_mean_latlon
        dayfrac_std[lat_idx,lon_idx] = dayfrac_std_latlon
        dayfrac_med[lat_idx,lon_idx] = dayfrac_med_latlon
        dayfrac_mad[lat_idx,lon_idx] = dayfrac_mad_latlon

        print('  year mean +/- std:', np.round(year_mean_latlon,1), np.round(year_std_latlon,1)) 
        print('  doy mean +/- std:', np.round(doy_mean_latlon,1), np.round(doy_std_latlon,1)) 
        print('    doy median +/- mad:', np.round(doy_med_latlon,1), np.round(doy_mad_latlon,1)) 
        print('  dayfrac mean +/- std:', np.round(dayfrac_mean_latlon,3), np.round(dayfrac_std_latlon,3))

0 43.25 250.5 4325N-25050E-debris_melt_curve.nc
  year mean +/- std: 2016.0 0.2
  doy mean +/- std: 22.1 13.2
    doy median +/- mad: 22.6 0.0
  dayfrac mean +/- std: 15.275 0.002


In [8]:
# Export to dataset
ds_ts_stats = xr.Dataset({'year_mean': (['latitude', 'longitude'], year_mean),
                          'year_std': (['latitude', 'longitude'], year_std),
                          'year_med': (['latitude', 'longitude'], year_med),
                          'year_mad': (['latitude', 'longitude'], year_mad),
                          'doy_mean': (['latitude', 'longitude'], doy_mean),
                          'doy_std': (['latitude', 'longitude'], doy_std),
                          'doy_med': (['latitude', 'longitude'], doy_med),
                          'doy_mad': (['latitude', 'longitude'], doy_mad),
                          'dayfrac_mean': (['latitude', 'longitude'], dayfrac_mean),
                          'dayfrac_std': (['latitude', 'longitude'], dayfrac_std),
                          'dayfrac_med': (['latitude', 'longitude'], dayfrac_med),
                          'dayfrac_mad': (['latitude', 'longitude'], dayfrac_mad),},
                          coords={'latitude': lat_values,
                                  'longitude': lon_values})

attrs_dict={
     'year_mean':{'units':'-',
         'long_name':'mean year',
         'comment': 'mean year when mosaicked surface temperature satellite image was acquired'},
     'year_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of year when mosaicked surface temperature satellite image was acquired'},
     'year_med':{'units':'-',
         'long_name':'median year',
         'comment': 'median year when mosaicked surface temperature satellite image was acquired'},
     'year_mad':{'units':'-',
         'long_name':'median absolute deviation year',
         'comment': 'median absolute deviation of year of when mosaicked surface temperature satellite image was acquired'},
     'doy_mean':{'units':'days since January 1',
         'long_name':'mean day of year',
         'comment': 'mean day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_std':{'units':'days since January 1',
         'long_name':'day of year standard deviation',
         'comment': 'standard deviation of day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_med':{'units':'days since January 1',
         'long_name':'median day of year',
         'comment': 'median day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_mad':{'units':'days since January 1',
         'long_name':'median absolute deviation day of year',
         'comment': 'day of year of year of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mean':{'units':'-',
         'long_name':'mean hour',
         'comment': 'mean hour of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_med':{'units':'-',
         'long_name':'median hour',
         'comment': 'median hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mad':{'units':'-',
         'long_name':'median absolute deviation hour',
         'comment': 'median absolute deviation of hour of when mosaicked surface temperature satellite image was acquired'},}

for vn in ['year_mean', 'year_std', 'year_med', 'year_mad',
           'doy_mean', 'doy_std', 'doy_med', 'doy_mad',
           'dayfrac_mean', 'dayfrac_std', 'dayfrac_med', 'dayfrac_mad',]:
    ds_ts_stats[vn].attrs = attrs_dict[vn]
    
ds_ts_stats.to_netcdf(ts_info_fullfn)
                
print(ds_ts_stats)

<xarray.Dataset>
Dimensions:       (latitude: 1, longitude: 1)
Coordinates:
  * latitude      (latitude) float64 43.25
  * longitude     (longitude) float64 250.5
Data variables:
    year_mean     (latitude, longitude) float64 2.016e+03
    year_std      (latitude, longitude) float64 0.1743
    year_med      (latitude, longitude) float64 2.016e+03
    year_mad      (latitude, longitude) float64 0.0
    doy_mean      (latitude, longitude) float64 22.1
    doy_std       (latitude, longitude) float64 13.18
    doy_med       (latitude, longitude) float64 22.57
    doy_mad       (latitude, longitude) float64 0.0
    dayfrac_mean  (latitude, longitude) float64 15.27
    dayfrac_std   (latitude, longitude) float64 0.002375
    dayfrac_med   (latitude, longitude) float64 15.27
    dayfrac_mad   (latitude, longitude) float64 0.0


In [9]:
# lat_idx = 4
# lon_idx = 2
# # lat_idx = 37
# # lon_idx = 46
# print(ds_ts_stats['year_mean'][lat_idx,lon_idx].values, ds_ts_stats['year_std'][lat_idx,lon_idx].values, 
#       ds_ts_stats['doy_mean'][lat_idx,lon_idx].values, ds_ts_stats['doy_std'][lat_idx,lon_idx].values)

In [10]:
ts_info_fullfn

'/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/16_ts_data/16_debris_tsinfo.nc'